# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Oprah Winfrey
- Miley Cyrus
- Sam Smith
- J. Cole

# Group Members IDs

- A########
- A########
- A########
- A########

# Research Question

*Fill in your research question here*

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
import pandas as pd
import json
import numpy as np

pd.set_option('precision', 2)

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

### Analysis 1

In [5]:
import import_ipynb
import city_and_county_datasets as citydata
import top500_undergradpop_demographic as undergrad_demo

df_undergrads = undergrad_demo.df_top500
df_cities = citydata.df_cities
df_cities

,African American,Asian,Hispanic,Multiracial,Native American,Pacific Islander,Unknown,White
cambridge-ma,0.11,0.16,0.09,0.04,0.00,0.00,2.00e-02,0.62
newhaven-ct,0.15,0.04,0.18,0.02,0.01,0.00,-2.43e-17,0.63
stanford-ca,0.04,0.27,0.14,0.10,0.00,0.00,2.00e-02,0.47
jacksonville-il,0.11,0.01,0.04,0.03,0.00,0.00,2.08e-17,0.82
westerville-oh,0.08,0.02,0.02,0.03,0.00,0.00,1.00e-02,0.85
whitewater-wi,0.04,0.02,0.10,0.03,0.00,0.01,2.00e-02,0.82
greensburg-pa,0.06,0.01,0.02,0.04,0.00,0.00,1.00e-02,0.86
redding-ca,0.01,0.05,0.10,0.05,0.02,0.00,3.00e-02,0.78
sanbernardino-ca,0.09,0.08,0.53,0.04,0.02,0.01,-1.00e-02,0.29
berkeley-ca,0.09,0.20,0.11,0.07,0.01,0.01,2.00e-02,0.55


In [6]:
df_undergrads

,ranking,name,African American,Asian,Hispanic,Multiracial,Native American,Pacific Islander,Unknown,White,undergradpop
0,1,massachusetts-institute-of-technology,0.07,0.28,0.17,0.07,0,0,0.02,0.39,4510
1,2,stanford-university,0.07,0.23,0.16,0.12,0.01,0,0.01,0.4,7064
2,3,harvard-university,0.07,0.22,0.12,0.06,0,0,0.04,0.49,7147
3,4,yale-university,0.07,0.19,0.14,0.06,0.01,0,0.04,0.49,5743
4,5,princeton-university,0.08,0.23,0.1,0.05,0,0,0.07,0.47,5260
5,6,university-of-pennsylvania,0.08,0.23,0.1,0.05,0,0,0.05,0.49,10496
6,7,columbia-university,0.09,0.21,0.16,0.06,0,0,0.04,0.45,7592
7,8,duke-university,0.11,0.23,0.09,0.02,0.01,0,0.03,0.51,6669
8,9,brown-university,0.07,0.16,0.13,0.07,0,0,0.09,0.48,6666
9,10,california-institute-of-technology,0.01,0.47,0.13,0.08,0,0,0,0.31,961


# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*